In [30]:
!pip install folium
import folium
import json
import os
import requests
import pandas
import numpy as np

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [38]:
arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table["race_new"] = arrest_table["race"]
arrest_table["sex_new"] = arrest_table["sex"]
arrest_table["race"] = arrest_table["race_new"]
arrest_table["sex"] = arrest_table["sex_new"]
arrest_table = arrest_table.drop('race_new', 1)
arrest_table = arrest_table.drop('sex_new', 1)

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

ndata = arrest_table.iloc[:640]
print(len(arrest_table.loc[arrest_table['sex'] == 'F']))
print(len(arrest_table.loc[arrest_table['sex'] == 'M']))


ndata

11369
52522


,arrest,age,race,sex,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,B,M,01/01/2011,00:01:00,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,B,M,01/01/2011,00:01:00,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,B,M,01/01/2011,00:04:00,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,B,M,01/01/2011,00:05:00,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,B,M,01/01/2011,00:05:00,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358
6,11126932.0,29,B,M,01/01/2011,00:05:00,800 N Monroe St,79-Other,800 N Monroe St,1 5212,Handgun On Person || Handgun Violation,WESTERN,724.0,Midtown-Edmondson,"(39.2979815407, -76.6475113571)",39.297982,-76.647511
7,11126940.0,20,W,M,01/01/2011,00:05:00,5200 Moravia Rd,Unknown Offense,NaN,1 5200,Deadly Weapon-Int/Injure || Aggravated Assault,NORTHEASTERN,436.0,Frankford,"(39.3235271620, -76.5496555072)",39.323527,-76.549656
9,11127018.0,53,B,M,01/01/2011,00:15:00,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,1 1425,Reckless Endangerment || Hgv,NORTHWESTERN,614.0,Central Park Heights,"(39.3436773374, -76.6727297618)",39.343677,-76.672730
10,11127057.0,28,B,M,01/01/2011,00:15:00,3300 Woodland Ave,54-Armed Person,3300 Woodland Av,1 1425,Reckless Endangerment || Hgv,NORTHWESTERN,614.0,Central Park Heights,"(39.3436773374, -76.6727297618)",39.343677,-76.672730
11,11126892.0,25,B,M,01/01/2011,00:20:00,2800 Violet Ave,79-Other,2800 Violet Av,1 5212,Handgun On Person || Hgv,NORTHWESTERN,612.0,Park Circle,"(39.3298283117, -76.6618069447)",39.329828,-76.661807


In [34]:
arrest_table.iloc[:640].sex.unique()

array(['M', 'F'], dtype=object)

In [42]:
map_osm = folium.Map(location=[39.29, -76.61], tiles='Stamen Toner', zoom_start=11)
tooltip = 'Click me!'
for index, row in ndata.iterrows():
    color=''
    if row['race'] == 'B':
        color='red'
    if row['race'] == 'W':
        color='orange'
    if row['race'] == 'A':
        color='yellow'
    if row['race'] == 'U':
        color='green'
    if row['race'] == 'I':
        color='blue'
    if row['race'] == 'H':
        color='purple'
    if row['race'] == np.nan:
        color='crimson'
    folium.Circle(
    radius=40,
    location=[row['lat'], row['long']],
    popup=row['incidentOffense'],
    color=color,
    fill=True,
).add_to(map_osm)
    

map_osm